In [1]:
# import libraries
import numpy as np
import pandas as pd

In [21]:
# load news file from kaggle
articles_left = pd.read_csv('..\\..\\data\\braindedleft.csv')
print(articles_left.shape)
articles_right = pd.read_csv('..\\..\\data\\braindedright.csv')
print(articles_right.shape)

(2411, 14)
(1810, 14)


(2411, 14)

,0
0,https://ki
1,https://ww
41,https://ap
48,https://bl
67,https://ab
...,...
2155,https://sp
2179,https://cb
2181,https://pb
2183,https://me
